In [4]:
import json
from json import JSONEncoder
import re
import time
import datetime

In [5]:
class SensorData:
    def __init__(self):
        self.date = '' # day for which data is stored
        self.sensor = {} # dictionary of parameter and List of values

In [6]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__ 
# http://stackoverflow.com/questions/3768895/how-to-make-a-class-json-serializable    

In [16]:
def parse_sensor_data_test(file_name):
    start_time = time.time()    
    
    PATTERN_HEADER = r'(?m)^(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})'
    PATTERN_DATA = r'(?m)(?P<temperature>\d{4}|NNNN)(?P<humidity>\d{3}|NNN)'
    
    sensor_data_regex = re.compile(PATTERN_DATA)
    
    all_data = []
    
    with open(file_name,'r', encoding='utf-8') as rdr:            
        for line in rdr:
            # One row contains a day worth of data
            sensor_data = SensorData()
            sensor_data.sensor['temperature'] = []
            sensor_data.sensor['humidity'] = []
            
            match = re.search(PATTERN_HEADER, line)

            if match:
                sensor_data.date = \
                    '-'.join([match.group('year'),match.group('month'),match.group('day')])
                
                header_len = len(match.group(0))
                
                # compiled object - you can search from specific point in text
                #  skip the header portion and look for temperature-humdity pairs
                match_iter = sensor_data_regex.finditer(line,header_len)
                
                for match in match_iter:
                    # access temperature and humidity groups
                    sensor_data.sensor['temperature'].append(match.group('temperature'))
                    sensor_data.sensor['humidity'].append(match.group('humidity'))                    

                all_data.append(sensor_data)
        
        print ('Elapsed Time For Parsing: {0:.2f}s'.format(time.time()-start_time))
        
        with open(file_name+'.json','w', encoding='utf-8') as wr:
            json.dump(all_data, wr, ensure_ascii=False, cls=MyEncoder, indent=True)
            
        print ('Elapsed Time After Write to File : {0:.2f}s'.format(time.time()-start_time))

In [22]:
file_name =  r"..\Data\SensorData\sensordata_7days.txt"
parse_sensor_data_test(file_name)

Elapsed Time For Parsing: 0.02s
Elapsed Time After Write to File : 0.06s
